In [6]:
import os
import re
import json
import pandas as pd
import re
from unidecode import unidecode
from sklearn.cluster import KMeans
from numbers import Number

folderPathBaseIdealista = '..\\..\\Bases\\idealista\\202508'
folderPathBaseFinal = '..\\..\\Bases\\idealista\\base_processada\\base_imoveis.csv'
folderPathBaseMunicipioFinal = '..\\..\\Bases\\idealista\\base_processada\\base_imoveis_municipio.csv'

def listar_jsons(raiz):
    arquivos_json = []
    for pasta_atual, subpastas, arquivos in os.walk(raiz):
        for arquivo in arquivos:
            if arquivo.endswith('.json'):
                caminho_completo = os.path.join(pasta_atual, arquivo)
                arquivos_json.append(caminho_completo)
    return arquivos_json

def ler_todos_jsons_para_dataframe(raiz):
    lista_df = []
    arquivos_json = listar_jsons(raiz)
    for caminho in arquivos_json:
        try:
            df = pd.read_json(caminho)
            df['arquivo_origem'] = os.path.basename(caminho)  # nome do arquivo
            df['caminho_pasta'] = os.path.dirname(caminho)    # caminho da pasta
            lista_df.append(df)
        except Exception as e:
            print(f"Erro ao ler {caminho}: {e}")
    if lista_df:
        df_final = pd.concat(lista_df, ignore_index=True)
    else:
        df_final = pd.DataFrame()

    return df_final

def trata_imoveis(df):
    # Aplica ao DataFrame inteiro:
    df['municipio'] = df['arquivo_origem'].apply(extrair_municipio)
    df['caminho_pasta'] = df['caminho_pasta'].str.replace(r"^\.\.\\\.\.\\Bases\\idealista\\202508\\", "", regex=True)
    df['preco_mes'] = df['Preco'].apply(tratar_preco)
    df['areaBrutaM2'] = df['areaBrutaM2'].replace(0, pd.NA).fillna(pd.NA)
    df['preco_metro_quadrado'] = df['preco_mes'] / df['areaBrutaM2']
    df['numero_andar'] = df['pisoResumo'].apply(extrair_numero_andar)
    df['tem_elevador'] = df['pisoResumo'].apply(extrair_elevador)
    df['qtd_quartos'] = df['tipologia'].str.replace('T', '').astype(int)
    df['cat_quartos'] = df['qtd_quartos'].apply(categoria_quartos)
    # df['Outlier_m2_municipio'] = df.groupby('municipio', group_keys=False).apply(rotular_outliers_por_municipio)
    df['Outlier_m2_municipio'] = df.groupby('municipio')['preco_metro_quadrado'].transform(rotular_outliers_por_municipio)
    colunas = [
        'arquivo_origem',
        'caminho_pasta',
        'municipio',
        'id',
        'Titulo',
        'detalhe do item',
        'Descricao do item',
        # 'estacionamento',
        # 'tipologia',
        'qtd_quartos',
        'cat_quartos',
        'numero_andar',
        'tem_elevador',
        # 'pisoResumo',
        'Link',
        'areaBrutaM2',
        # 'Preco',
        'preco_mes',
        'preco_metro_quadrado',
        'Outlier_m2_municipio',
        # 'precoOriginal',
        # 'descontoPercentual',
        # 'tempoDestaque',
        'imagens',
        'tags'

    ]
    
    df = df.reindex(columns=colunas)
    df = trata_tags(df)
    df = df.dropna(axis=1, how='all')

    return df

def extrair_municipio(nome_arquivo):
    # Pega tudo antes de "-Paginas"
    resultado = re.match(r"^(.*?)-Paginas", nome_arquivo)
    if resultado:
        return resultado.group(1)
    else:
        return nome_arquivo.replace('.json', '')  # fallback: tira o .json
    
def tratar_preco(preco):

    if pd.isna(preco):
        return pd.NA

    if isinstance(preco, Number):
        return pd.to_numeric(preco, errors="coerce")

    preco_str = str(preco)
    preco_limpo = re.sub(r"[^0-9.,]", "", preco_str)

    if "," in preco_limpo and "." in preco_limpo:
        preco_limpo = preco_limpo.replace(".", "").replace(",", ".")
    elif "," in preco_limpo:
        preco_limpo = preco_limpo.replace(",", ".")
    else:
        preco_limpo = preco_limpo.replace(".", "")

    return pd.to_numeric(preco_limpo, errors="coerce")
    
def extrair_numero_andar(piso):
    if pd.isnull(piso):
        return None
    # Checa por 'Rés do chão' ou 'Cave'
    if "rés do chão" in piso.lower():
        return 0
    if "cave" in piso.lower():
        return -1
    
    # Novo padrão para 'Andar -2', 'Andar -1', etc
    match_andar_neg = re.search(r"andar\s*([-+]?\d+)", piso.lower())
    if match_andar_neg:
        return int(match_andar_neg.group(1))
    
    # Busca padrão de número
    match = re.search(r"(\d+)[ºo]? andar", piso.lower())
    if match:
        return int(match.group(1))
    return None

def extrair_elevador(piso):
    if pd.isnull(piso):
        return None
    piso_lower = piso.lower()
    if "com elevador" in piso_lower:
        return 1
    if "sem elevador" in piso_lower:
        return 0
    # Se só tem "elevador" e não especificou com/sem, marca como True
    if "elevador" in piso_lower:
        return 1
    return None

def categoria_quartos(q):
    categorias = {
            0: "0.Quarto",
            1: "1.Quarto",
            2: "2.Quartos",
            3: "3.Quartos",
            4: "4.Quartos",
        }

    if pd.isna(q) or q < 0:
        return None

    return categorias.get(q, ">5.Quartos")

def trata_tags(df):
    # Explode as tags em linhas
    df_exploded = df.explode('tags')
    # Gera as colunas de dummies (1 para cada tag)
    dummies = pd.get_dummies(df_exploded['tags'])

    # Junta com o id e agrupa pelo id, pegando o máximo (se tiver a tag, vira 1)
    df_tags = pd.concat([df_exploded['id'], dummies], axis=1).groupby('id').max().reset_index()

    tag_cols = dummies.columns
    df_tags[tag_cols] = df_tags[tag_cols].astype(int)

    # Junta de volta no dataframe original (se quiser)
    df = pd.merge(df, df_tags, on='id')
    
    return df

def formatar_numericos(df):
        df = df.copy()  # Cria uma cópia, o original não será alterado!
        for col in df.select_dtypes(include='number').columns:
            df[col] = df[col].apply(lambda x: '{:,.2f}'.format(x).replace(',', 'X').replace('.', ',').replace('X', '.'))
        return df

def rotular_outliers_por_municipio(g: pd.DataFrame) -> pd.Series:
    s = g['preco_metro_quadrado']
    # menos de 8 valores válidos → NA
    if s.notna().sum() < 8:
        return pd.Series([pd.NA]*len(g), index=g.index, dtype='Int64')

    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr

    mask = (s < lim_inf) | (s > lim_sup)          # caro/baixo demais
    out = mask.astype('Int64')                     # True/False → 1/0
    out[s.isna()] = pd.NA                          # mantém NaN onde o preço é NaN
    return out

def trata_municipios(df_imoveis):
    pivot = pd.pivot_table(
        df_imoveis,
        index='municipio',
        columns='cat_quartos',
        values='preco_mes',
        aggfunc=['mean', 'median'],
        fill_value=0
    )

    # Ajustando nome das colunas
    pivot.columns = [f'{func}.{col}' for func, col in pivot.columns]

    # Ordenando as colunas para ficar na sequência desejada
    ordem = [
        'mean.0.Quarto','mean.1.Quarto', 'mean.2.Quartos', 'mean.3.Quartos', 'mean.4.Quartos', 'mean.>5.Quartos',
        'median.0.Quarto','median.1.Quarto', 'median.2.Quartos', 'median.3.Quartos', 'median.4.Quartos', 'median.>5.Quartos'
    ]
    pivot = pivot.reindex(columns=ordem, fill_value=0)

    pivot = pivot.round(2)

    # Ajuste removendo outliers
    df_imoveis_ajustado = df_imoveis.where(df_imoveis['Outlier_m2_municipio'] != 1, pd.NA)
    
    # Médias do preço por categoria de quartos
    medias_quartos = df_imoveis_ajustado.pivot_table(
        index='municipio',
        columns='cat_quartos',
        values='preco_mes',
        aggfunc='mean'
    )
    # Renomeia as colunas para identificar como média
    medias_quartos = medias_quartos.add_prefix('media_preco_').reset_index()

    df_moveis_municipio = df_imoveis_ajustado.groupby(['caminho_pasta','municipio']).agg(
        # Quantidades
        qtd_imoveis = ('id','count'),
        qtd_imoveis_T0 = ('qtd_quartos', lambda x: (x == 0).sum()),
        qtd_imoveis_T1 = ('qtd_quartos', lambda x: (x == 1).sum()),
        qtd_imoveis_T2 = ('qtd_quartos', lambda x: (x == 2).sum()),
        qtd_imoveis_T3 = ('qtd_quartos', lambda x: (x == 3).sum()),
        qtd_imoveis_T4 = ('qtd_quartos', lambda x: (x == 4).sum()),
        qtd_imoveis_T5_mais = ('qtd_quartos', lambda x: (x >= 5).sum()),

        # Valores
        media_valor_arrendamento = ('preco_mes', 'mean'),
        mediana_valor_arrendamento = ('preco_mes', 'median'),
        max_valor_arrendamento = ('preco_mes', 'max'),
        min_valor_arrendamento = ('preco_mes', 'min'),
        sdt_valor_arrendamento = ('preco_mes', 'std'),

        media_valor_metro_quadrado = ('preco_metro_quadrado', 'mean'),
        mediana_valor__metro_quadrado = ('preco_metro_quadrado', 'median'),
        max_valor_metro_quadrado = ('preco_metro_quadrado', 'max'),
        min_valor_metro_quadrado = ('preco_metro_quadrado', 'min'),
        std_valor_metro_quadrado = ('preco_metro_quadrado', 'std'),

        # Area
        media_metro_quadrado = ('areaBrutaM2', 'mean'),
        mediana_metro_quadrado  = ('areaBrutaM2', 'median'),
        max_metro_quadrado = ('areaBrutaM2', 'max'),
        min_metro_quadrado = ('areaBrutaM2', 'min'),
        std_metro_quadrado = ('areaBrutaM2', 'std'),

    ).reset_index()

    # join com o pivot
    df_moveis_municipio = pd.merge(df_moveis_municipio, pivot, how='left', on='municipio')
    # join com as médias por categoria de quartos
    df_moveis_municipio = pd.merge(df_moveis_municipio, medias_quartos, how='left', on='municipio')

    df_moveis_municipio.rename(columns={
        'caminho_pasta': 'distrito',
        'municipio': 'municipio',
        'qtd_imoveis': 'qtd.imoveis',
        'media_valor_arrendamento': 'media.valor.arrendamento',
        'mediana_valor_arrendamento': 'mediana.valor.arrendamento',
        'max_valor_arrendamento': 'max.valor.arrendamento',
        'min_valor_arrendamento': 'min.valor.arrendamento',
        'sdt_valor_arrendamento': 'sdt.valor.arrendamento',
        'media_valor_metro_quadrado': 'media.valor.metro.quadrado',
        'mediana_valor__metro_quadrado': 'mediana.valor.metro.quadrado',
        'max_valor_metro_quadrado': 'max.valor.metro.quadrado',
        'min_valor_metro_quadrado': 'min.valor_metro.quadrado',
        'std_valor_metro_quadrado': 'std.valor.metro.quadrado',
        'media_metro_quadrado': 'media.metro.quadrado',
        'mediana_metro_quadrado': 'mediana.metro.quadrado',
        'max_metro_quadrado': 'max.metro.quadrado',
        'min_metro_quadrado': 'min.metro.quadrado',
        'std_metro_quadrado': 'std.metro.quadrado',
        'qtd_imoveis_T0': 'qtd.imoveis.T0',
        'qtd_imoveis_T1': 'qtd.imoveis.T1',
        'qtd_imoveis_T2': 'qtd.imoveis.T2',
        'qtd_imoveis_T3': 'qtd.imoveis.T3',
        'qtd_imoveis_T4': 'qtd.imoveis.T4',
        'qtd_imoveis_T5_mais': 'qtd.imoveis.T5.mais',
        'mean.0.Quarto': 'valor.medio.0.Quarto',
        'mean.1.Quarto': 'valor.medio.1.Quarto',
        'mean.2.Quartos': 'valor.medio.2.Quartos',
        'mean.3.Quartos': 'valor.medio.3.Quartos',
        'mean.4.Quartos': 'valor.medio.4.Quartos',
        'mean.5.ou.mais.Quartos': 'valor.medio.5.ou.mais.Quartos',
        'median.0.Quarto': 'valor.mediano.0.Quarto',
        'median.1.Quarto': 'valor.mediano.1.Quarto',
        'median.2.Quartos': 'valor.mediano.2.Quartos',
        'median.3.Quartos': 'valor.mediano.3.Quartos',
        'median.4.Quartos': 'valor.mediano.4.Quartos',
        'median.5.ou.mais.Quartos': 'valor.mediano.5.ou.mais.Quartos'
    }, inplace=True)

    return df_moveis_municipio

def salva_base(df, filePath):
    df.to_csv(filePath, index=False)


In [7]:
# Exemplo de uso:
df_total = ler_todos_jsons_para_dataframe(folderPathBaseIdealista)
df_imoveis = trata_imoveis(df_total)

KeyError: 'preco_metro_quadrado'

In [ ]:
base = formatar_numericos(df_imoveis)
salva_base(base, folderPathBaseFinal)


In [ ]:
df_moveis_municipio = trata_municipios(df_imoveis)

In [ ]:
base_municipio = formatar_numericos(df_moveis_municipio)
salva_base(base_municipio, folderPathBaseMunicipioFinal)
